# Computational Modeling Across the Blood-Brain Barrier
#### Group - ALEX 
#### Members - Alex Luna, Alex Dhupar, Alexis Guth
***

## Background Information
***

The goal of this project is to analyze data regarding current drugs and their ability to cross the blood brain barrier, with the eventual aim of creating an equation based on the analyzed data in order to generate a predictive model. This model will be based on drugs which have the ability to cross the Blood Brain Barrier (BBB), and their chemical and physical properties. This information will be used to generate a mathematical equation which will be able to graphically determine a drug's ability to cross the BBB based on chosen factors. 

The package rdkit was used to pull elements of interest out of the molecules provided, based on their SMILES formula, which is a computer friendly chemical formula for a molecule of interest. The elements of interest will be analyzed are: Lipophilicity (MolLogP), Molecular Weight (MolWt), Topological Polar Surface Area (TPSA), Formal Charge (FC), Aromatic Rings, and Heavy atoms. 

- These elements were chosen specifically for their impact on the ability for a drug to cross a membrane. Lipophilicity was chosen.....

These physiochemical elements will then be used to find any correlation between an element and the LogBB value, which is an experimental value given in the Data set. LogBB describes the concentration of a drug found on either side of the BBB, where a higher LogBB value equals a higher concentration of drug found on the inside of the BBB. 

The data set used in this Project was sourced from GitHub, and includes data of ~7000 drugs and their ability to cross the BBB. The specific data set used in the following data analysis is comprised of only ~1000 drugs, as this specific set contains the LogBB values. 
‌

#### References
[1] F. Meng, Y. Xi, J. Huang, and P. W. Ayers, “A curated diverse molecular database of blood-brain barrier permeability with chemical descriptors,” Scientific Data, vol. 8, no. 1, p. 289, Oct. 2021, doi: https://doi.org/10.1038/s41597-021-01069-5.

## Data Preparation and Feature Extraction
***

In [1]:
# import required packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Descriptors

Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'rdkit'

In [ ]:
def ecdf(data):
    """Generate x and y values for plotting an ECDF."""
    x_vals = np.sort(data)
    y_vals = np.arange(1, len(data)+1) / len(data)

    return x_vals, y_vals

In [ ]:
# data file name for BBB dataset with logBB values
bbb_reg = "../B3DB/B3DB_regression.tsv"

# load data
df_regression = pd.read_csv(bbb_reg, sep="\t")

In [ ]:
df_regression.head()

In [ ]:
# remove any variables that do not have a compound name
df_main = df_regression[(df_regression['compound_name'].notna())].copy()

# parse through molecular structures and prepare for structure analysis
df_main['Mol'] = df_main['SMILES'].apply(Chem.MolFromSmiles)

# based on molecular structures, add new columns containing elements of interest
df_main['MolLogP'] = df_main['Mol'].apply(Descriptors.MolLogP)
df_main['MolWt'] = df_main['Mol'].apply(Descriptors.MolWt)
df_main['TPSA'] = df_main['Mol'].apply(Descriptors.TPSA)
df_main['FC']    = df_main['Mol'].apply(lambda m: Chem.GetFormalCharge(m))
df_main['Aromatic Rings'] = df_main['Mol'].apply(Descriptors.NumAromaticRings)
df_main['Heavy Atoms'] = df_main['Mol'].apply(Descriptors.HeavyAtomCount)


In [ ]:
# ensure data frame only consists of elements of interest
df_main = df_main[['NO.', 'compound_name', 'logBB', 'MolLogP', 'MolWt',
                   'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']]
df_main.head()

## Exploratory Data Analysis
***

In [ ]:
# initialize list of elements and plotting setup
elements = ['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']
fig, axes = plt.subplots(3,2, figsize = (10,10))
axes = axes.flatten()

# loop through elements
for i,elm in enumerate(elements):

    # plotting our data
    x_data, y_data = ecdf(df_main[elm])
    axes[i].scatter(x_data, y_data)
    axes[i].set_xlabel(elm)
    axes[i].set_ylabel('ECDF');

In [ ]:
# initialize list of elements and plotting setup
elements = ['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']
fig, axes = plt.subplots(3,2, figsize = (10,10))
axes = axes.flatten()

# loop through elements
for i, elm in enumerate(elements):

    # separate data of element into bins
    bins = df_main[elm].unique().tolist()
    bins.sort()

    # develop count for each bin
    count = []
    for b in bins:
        count.append((df_main[elm] == b).sum())

    # plot data
    axes[i].bar(bins, count)
    axes[i].set_xlabel(f'Number of {elm}')
    axes[i].set_ylabel('Count')

# title graph and ensure proper spacing
axes[4].set_xlabel('Molecules with MolWt')
axes[5].set_xlabel('Molecules with TPSA')

fig.suptitle('Bar Charts for Visualization of Element counts')
plt.tight_layout();

In [ ]:
# initialize list of elements and plotting setup
elements = ['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']
fig, axes = plt.subplots(3,2, figsize = (10,10))
axes = axes.flatten()

# loop through elements
for i,elm in enumerate(elements):

    # plot data
    axes[i].scatter(df_main['logBB'], df_main[elm])
    axes[i].set_xlabel('logBB')
    axes[i].set_ylabel(elm)    


# title graph and ensure proper spacing
fig.suptitle('Scatterplot for Correlation between LogBB and Elements')
plt.tight_layout();

In [ ]:
# make a list of elements
elements = ['logBB', 'MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']

# make a correlation table between all chosen elements
corr_table = df_main[elements].corr()

# format table to include a gradient
corr_table.style.background_gradient(cmap='coolwarm') \
                 .format("{:.2f}")


## Regression Modelling
***

#### Simple Linear Regression

In [ ]:
# import additional packages
from scipy.stats import linregress

# make a list of elements
elements = ['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']

# set up subplot grid
fig, axes = plt.subplots(3, 2, figsize=(12, 12))
axes = axes.flatten()

# loop through descriptors
for i, elm in enumerate(elements):
    x = df_main[elm]
    y = df_main['logBB']
    
    # perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    
    # scatter plot
    axes[i].scatter(x, y, alpha=0.6, label='Data')
    
    # regression line
    axes[i].plot(x, intercept + slope*x, color='red', label=f'Fit (R²={r_value**2:.2f})')
    
    # labels and title
    axes[i].set_xlabel(elm)
    axes[i].set_ylabel('logBB')
    axes[i].set_title(f'{elm} vs logBB')
    axes[i].legend()

# overall title and layout
fig.suptitle('Linear Regression of logBB vs Molecular Descriptors', fontsize=16)
plt.tight_layout()
plt.show()

#### Multiple Linear Regression

In [ ]:
# select features and target
X = df_main[['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']].values
y = df_main['logBB'].values

# add intercept column
X_aug = np.column_stack((np.ones(X.shape[0]), X))

# solve using least squares
coeffs, residuals, rank, s = np.linalg.lstsq(X_aug, y, rcond=None)

# print coefficients
print("Intercept:", coeffs[0])
for i, name in enumerate(['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']):
    print(f"{name}: {coeffs[i+1]:.4f}")

In [ ]:
feature_names = ['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']
coef_values = coeffs[1:]  # skip intercept

# plot regression coefficients to show which descriptors matter most
plt.figure(figsize=(6,4))
plt.bar(feature_names, coef_values)
plt.xticks(rotation=45)
plt.ylabel('Coefficient Value')
plt.title('Multiple Regression Coefficients')
plt.tight_layout()
plt.show()

In [ ]:
# predicted values
y_pred = X_aug @ coeffs

# plot predicted vs actual
plt.figure(figsize=(6,6))
plt.scatter(y, y_pred, alpha=0.6)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', label='Ideal fit')
plt.xlabel('Actual logBB')
plt.ylabel('Predicted logBB')
plt.title('Multiple Linear Regression: Predicted vs Actual logBB')
plt.legend()
plt.tight_layout()
plt.show()

#### Logisitic Regression and Classification

In [ ]:
# prepare features and binary target
X = df_main[['MolLogP', 'MolWt', 'TPSA', 'FC', 'Aromatic Rings', 'Heavy Atoms']].values
y = (df_main['logBB'] > 0).astype(int).values   # 1 if penetrates BBB, else 0

# add intercept column
X_aug = np.column_stack((np.ones(X.shape[0]), X))

# define sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# gradient descent for logistic regression
def logistic_regression(X, y, lr=0.001, epochs=10000):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)

    for _ in range(epochs):
        linear_model = np.dot(X, weights)
        y_pred = sigmoid(linear_model)

        # gradient
        gradient = np.dot(X.T, (y_pred - y)) / n_samples
        weights -= lr * gradient

    return weights

# train model
weights = logistic_regression(X_aug, y, lr=0.001, epochs=5000)

# predictions
y_pred_prob = sigmoid(np.dot(X_aug, weights))
y_pred_class = (y_pred_prob >= 0.5).astype(int)

# accuracy
accuracy = (y_pred_class == y).mean()
print("Accuracy:", accuracy)

# create confusion matrix, which breaks down correct vs. incorrect predictions
def confusion_matrix(y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    return np.array([[tn, fp],[fn, tp]])

cm = confusion_matrix(y, y_pred_class)
print("Confusion Matrix:\n", cm)

# visualize confusion matrix
fig, ax = plt.subplots(figsize=(4,4))
im = ax.imshow(cm, cmap='Blues')

ax.set_xticks([0,1])
ax.set_yticks([0,1])
ax.set_xticklabels(['No BBB', 'Yes BBB'])
ax.set_yticklabels(['No BBB', 'Yes BBB'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Manual Logistic Regression)')

# annotate counts
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], ha='center', va='center', color='black')

plt.tight_layout()
plt.show()

## Predictive Machine Learning Model
***

#### Predictive Model

In [ ]:
# predictive ML model using gradient boosting
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_absolute_error

# features and target
X = df_main[['MolLogP','MolWt','TPSA','FC','Aromatic Rings','Heavy Atoms']]
y = df_main['logBB']

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train model
gbr = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, max_depth=3, random_state=42)
gbr.fit(X_train, y_train)

# evaluate
y_pred = gbr.predict(X_test)
print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))

#### Interactive Predictive Model

In [ ]:
import holoviews as hv
import panel as pn

hv.extension('bokeh')

# sliders
MolLogP = pn.widgets.FloatSlider(name="MolLogP", start=-2, end=7, step=0.1, value=2.0)
MolWt   = pn.widgets.IntSlider(name="MolWt", start=50, end=900, step=5, value=350)
TPSA    = pn.widgets.IntSlider(name="TPSA", start=0, end=220, step=1, value=70)
FC      = pn.widgets.IntSlider(name="Formal Charge", start=-3, end=3, step=1, value=0)
Arom    = pn.widgets.IntSlider(name="Aromatic Rings", start=0, end=10, step=1, value=1)
Heavy   = pn.widgets.IntSlider(name="Heavy Atoms", start=5, end=100, step=1, value=30)

# prediction function
def predict_plot(MolLogP, MolWt, TPSA, FC, Arom, Heavy):
    X = np.array([[MolLogP, MolWt, TPSA, FC, Arom, Heavy]])
    pred = gbr.predict(X)[0]
    K = 10**pred
    data = pd.DataFrame({"Compartment":["Blood","Brain"], "Concentration":[1.0, K]})
    return hv.Bars(data, kdims="Compartment", vdims="Concentration").opts(
        title=f"Predicted LogBB={pred:.3f}, Brain/Blood Ratio={K:.2f}",
        width=400, height=300)

interactive = pn.bind(predict_plot, MolLogP, MolWt, TPSA, FC, Arom, Heavy)
pn.Column(MolLogP, MolWt, TPSA, FC, Arom, Heavy, interactive)

## Model Comparison and Interpretation
***

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score

# linear regression (continuous logBB prediction)
linreg = LinearRegression().fit(X_train, y_train)
print("Linear Regression R²:", r2_score(y_test, linreg.predict(X_test)))

# logistic regression (binary classification: logBB > 0)
y_class = (y > 0).astype(int)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_class, test_size=0.2, random_state=42)
logreg = LogisticRegression(max_iter=500).fit(X_train_c, y_train_c)
print("Logistic Regression Accuracy:", accuracy_score(y_test_c, logreg.predict(X_test_c)))

# predictive model (non-linear)
y_pred_gbr = gbr.predict(X_test)
print("Gradient Boosting R²:", r2_score(y_test, y_pred_gbr))

# feature importance from gradient boosting
plt.bar(X.columns, gbr.feature_importances_)
plt.xticks(rotation=45)
plt.ylabel("Importance")
plt.title("Feature Importance (Gradient Boosting)")
plt.show()

## Fick's Law Simulation
***

In [ ]:
import holoviews as hv
import panel as pn
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

hv.extension('bokeh')

# dataset
df_main = df_main.copy()

# sliders for chosen descriptors and drug selection dropdown
MolLogP = pn.widgets.FloatSlider(name="MolLogP", start=-2, end=7, step=0.1, value=2.0)
MolWt   = pn.widgets.IntSlider(name="MolWt", start=50, end=900, step=5, value=350)
TPSA    = pn.widgets.IntSlider(name="TPSA", start=0, end=220, step=1, value=70)
FC      = pn.widgets.IntSlider(name="Formal Charge", start=-3, end=3, step=1, value=0)
Arom    = pn.widgets.IntSlider(name="Aromatic Rings", start=0, end=10, step=1, value=1)
Heavy   = pn.widgets.IntSlider(name="Heavy Atoms", start=5, end=100, step=1, value=30)

drug_select = pn.widgets.Select(name="Choose Drug", options=list(df_main.index.astype(str)))

def fick_simulation(MolLogP, MolWt, TPSA, FC, Arom, Heavy, drug_select):
    '''Fick’s law simulation of blood–brain barrier transport.'''
    # when drug is chosen, override sliders with values
    if drug_select in df_main.index.astype(str):
        row = df_main.loc[int(drug_select)]
        MolLogP, MolWt, TPSA, FC, Arom, Heavy = row[['MolLogP','MolWt','TPSA','FC','Aromatic Rings','Heavy Atoms']]
        pred_logBB = row['logBB']  # or use your trained model
    else:
        # predict logBB from regression model
        X = np.array([[MolLogP, MolWt, TPSA, FC, Arom, Heavy]])
        pred_logBB = linreg.predict(X)[0]

    K = 10**pred_logBB
    P_eff = 0.15
    T_max = 24
    y0 = [1.0, 0.0]

    def rhs(t, y):
        Cb, Cbr = y
        dCb = -P_eff * (Cb - Cbr/K)
        dCbr =  P_eff * (Cb - Cbr/K)
        return [dCb, dCbr]

    sol = solve_ivp(rhs, (0, T_max), y0, t_eval=np.linspace(0, T_max, 200))
    df = pd.DataFrame({"t": sol.t, "Blood": sol.y[0], "Brain": sol.y[1]})

    curve = hv.Curve((df["t"], df["Blood"]), label="Blood") * hv.Curve((df["t"], df["Brain"]), label="Brain")
    curve = curve.opts(title=f"Fick's Law Simulation (LogBB={pred_logBB:.2f}, K={K:.2f})",
                       xlabel="Time (h)", ylabel="Concentration (normalized)", width=500, height=400)
    return curve

# bind interactive plot
interactive_plot = pn.bind(fick_simulation, MolLogP, MolWt, TPSA, FC, Arom, Heavy, drug_select)

# layout
pn.Column(pn.Row(MolLogP, MolWt, TPSA), pn.Row(FC, Arom, Heavy), drug_select, interactive_plot)

## Results and Discussion
***

### Results

#### Regression Equation

In [ ]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression().fit(X_train, y_train)

# coefficients and intercept
coeffs = dict(zip(X.columns, linreg.coef_))
intercept = linreg.intercept_

print("Intercept:", intercept)
print("Coefficients:", coeffs)

Using the above intercept and coefficients taken from the multiple regression model, a regression equation that should reasonably predict LogBB was determined to be the following:
$$ LogBB = $$

### Discussion

## Next Steps 
***